# Redis JSON Tutorial

## Setup

In [ ]:
%%capture
%pip install redis rich tqdm 

In [ ]:
import redis
from rich.pretty import pprint

In [ ]:
# creating cluster
# make sure jupyter server is connected to redis network
# `docker network connect redis_default jupyter-jupyter-1`
r = redis.RedisCluster(host='master', port=6379)

## Introduction to Redis JSON

The JSON capability of Redis Stack provides JavaScript Object Notation (JSON) support for Redis, which allows Redis to function as a document database.
It lets you store, update, and retrieve JSON values in a Redis database, similar to any other Redis data type. Redis JSON also works seamlessly with Search and Query to let you index and query JSON documents.

Primary features include:

- Full support for the JSON standard.
- [JSONPath](https://goessner.net/articles/JsonPath/) syntax for selecting/updating elements inside documents.
- Documents are stored as binary data in a tree structure, allowing fast access to sub-elements.
- Typed atomic operations for all JSON value types.

### Prerequisites

[Redis Stack](https://redis.io/downloads/?utm_source=redisinsight&utm_medium=app&utm_campaign=json_tutorial) >=7.2.0-v7 \
OR \
[RedisJSON](https://github.com/RedisJSON/RedisJSON/) >=2.6.8 \
OR \
A free Redis Stack instance on [Redis Cloud](https://redis.io/try-free/?utm_source=redisinsight&utm_medium=app&utm_campaign=json_tutorial).

## Creating JSON Documents

Here's a query that creates a JSON document describing a single bike.

In [ ]:
# Create a JSON document
bicycle_1 = {
    "model": "Jigger",
    "brand": "Velorim",
    "price": 270,
    "type": "Kids bikes",
    "specs": {
        "material": "aluminium",
        "weight": "10"
    },
    "description": "The Jigger is the best ride for the smallest of tikes!",
    "addons": [
        "reflectors",
        "grip tassles"
    ],
    "helmet_included": False
}

response = r.json().set('bicycle:1', '$', bicycle_1)
print(response)

Now retrieve the newly created JSON document.

In [ ]:
# Retrieve bicycle:1
result = r.json().get('bicycle:1')
pprint(result)

In the above example, the path, which is root (`$`), is implied. You could also write this command as:

```python
r.json().get('bicycle:1', '$')
```

You can also retrieve parts of documents using JSONPath expressions. JSONPath will be discussed in more detail later in this tutorial, but here are a few examples:

In [ ]:
# Get the price of bicycle:1
price = r.json().get('bicycle:1', '$.price')
print(f"Price: {price}")

In [ ]:
# Get the weight of bicycle:1
weight = r.json().get('bicycle:1', '$.specs.weight')
print(f"Weight: {weight}")

In [ ]:
# Get the first addon of bicycle:1
first_addon = r.json().get('bicycle:1', '$.addons[0]')
print(f"First addon: {first_addon}")

There are two other commands you can use to get information from documents:

In [ ]:
# Get the length of bicycle:1's description
length = r.json().strlen('bicycle:1', '$.description')
print(f"Description length: {length}")

In [ ]:
# Get the type of bicycle:1's helmet_included attribute
type_value = r.json().type('bicycle:1', '$.helmet_included')
print(f"Helmet included type: {type_value}")

## Modifying JSON Documents

Modifying JSON documents is straightforward using a combination of JSONPath expressions and Redis's JSON command set.

Here are some examples.

### Extend documents

In [ ]:
# Add two more documents 
bicycle_2 = {
    "model": "Hillcraft", 
    "brand": "Bicyk", 
    "price": 1200, 
    "type": "Kids Mountain Bikes", 
    "specs": {
        "material": "carbon", 
        "weight": "11"
    }, 
    "description": "A light mountain bike for kids.", 
    "addons": ["reflectors", "safety lights"],
    "helmet_included": False
}

bicycle_3 = {
    "model": "Chook air 5", 
    "brand": "Nord", 
    "price": 815, 
    "type": "Kids Mountain Bikes", 
    "specs": {
        "material": "alloy", 
        "weight": "9.1"
    }, 
    "description": "A lighter, more durable mountain bike for six years and older.", 
    "addons": ["reflectors", "safety lights"],
    "helmet_included": False
}

r.json().set('bicycle:2', '$', bicycle_2)
r.json().set('bicycle:3', '$', bicycle_3)
print("Documents created successfully")

In [ ]:
# Add a new name-value pair to an existing document
response = r.json().set('bicycle:1', '$.newmember', 'value')
print(response)

# Check the updated document
result = r.json().get('bicycle:1')
pprint(result)

You could also update multiple documents at the same time. First, delete `$.newmember` from `bicycle:1` using `JSON.DEL`.

In [ ]:
# Delete $.newmember from bicycle:1
response = r.json().delete('bicycle:1', '$.newmember')
print(f"Number of paths deleted: {response}")

# Verify deletion
result = r.json().get('bicycle:1')
pprint(result)

Next, add `$.newmember` to all three bicycles:

In [ ]:
# Add a member named newmember to all three bicycles
r.json().set('bicycle:1', '$.newmember', 'value1')
r.json().set('bicycle:2', '$.newmember', 'value2')
r.json().set('bicycle:3', '$.newmember', 'value3')
print("New members added successfully")

# Check the new members
results = [r.json().get(key, '$.newmember') for key in ['bicycle:1', 'bicycle:2', 'bicycle:3']]
for idx, result in enumerate(results):
    print(f"Bicycle {idx+1} newmember: {result}")

### Manipulate numeric values

The `JSON.NUMINCRBY` command allows you to perform arithmetic operations on numeric fields of documents.
Use positive numbers to increment and negative numbers to decrement.

In [ ]:
# Decrease the price of bicycle:1
print(f"Original price: {r.json().get('bicycle:1', '$.price')}")

new_price = r.json().numincrby('bicycle:1', '$.price', -10)
print(f"New price after decreasing by 10: {new_price}")

print(f"Verified price: {r.json().get('bicycle:1', '$.price')}")

### Manipulate string and boolean values

Appending information to JSON strings is straightforward.

In [ ]:
# Append a string to bicycle:1's model
print(f"Original model: {r.json().get('bicycle:1', '$.model')}")

length = r.json().strappend('bicycle:1', 'naut', '$.model')
print(f"New length: {length}")

print(f"New model after appending: {r.json().get('bicycle:1', '$.model')}")

The `JSON.TOGGLE` command can be used to toggle boolean values.

In [ ]:
# Toggle the value of bicycle:1's helmet_included value
print(f"Original helmet_included value: {r.json().get('bicycle:1', '$.helmet_included')}")

new_value = r.json().toggle('bicycle:1', '$.helmet_included')
print(f"New value after toggle: {new_value}")

print(f"Verified helmet_included value: {r.json().get('bicycle:1', '$.helmet_included')}")

### Deeper document manipulation

As you saw earlier, the `JSON.MERGE` command can be used to create new documents. Additionally, it can also be used for the following use cases:

- Create a non-existant path-value:

In [ ]:
# Add a new name-value pair to bicycle:1
response = r.json().merge('bicycle:1', '$.newmember2', 'value 2 1')
print(response)

# Check the updated document
result = r.json().get('bicycle:1')
pprint(result)

- Replace an existing value:

In [ ]:
# Change bicycle:1's model back to Jigger
response = r.json().merge('bicycle:1', '$.model', 'Jigger')
print(response)

# Check the model
model = r.json().get('bicycle:1', '$.model')
print(f"Updated model: {model}")

- Delete an existing value:

In [ ]:
# Delete newmember2 from bicycle:1
response = r.json().merge('bicycle:1', '$', {"newmember2": None})
print(response)

# Check the updated document
result = r.json().get('bicycle:1')
pprint(result)

- Replace an array:

In [ ]:
# Replace bicycle:1's addons
response = r.json().merge('bicycle:1', '$.addons', ["reflectors", "rainbow seat"])
print(response)

# Check the updated addons
addons = r.json().get('bicycle:1', '$.addons')
print(f"Updated addons: {addons}")

- Make changes to multiple paths (no example).

### Delete information

You can delete name-value pairs using the `JSON.DEL` or `JSON.FORGET` commands:

In [ ]:
# Delete newmember from bicycle:1
response = r.json().delete('bicycle:1', '$.newmember')
print(f"Number of paths deleted: {response}")

# Check the updated document
result = r.json().get('bicycle:1')
pprint(result)

The `JSON.CLEAR` command will empty all arrays and set all numeric values to zero. A simple example will illustrate how this works.

In [ ]:
# JSON.CLEAR usage
doc = {
    "obj": {"a": 1, "b": 2}, 
    "arr": [1, 2, 3], 
    "str": "foo", 
    "bool": True, 
    "int": 42, 
    "float": 3.14
}

# Create the document
response = r.json().set('doc', '$', doc)
print(f"Document creation response: {response}")

# Check the original document
print("Original document:")
result = r.json().get('doc', '$')
pprint(result)

# Clear all values
cleared = r.json().clear('doc', '$.*')
print(f"\nNumber of paths cleared: {cleared}")

# Check the cleared document
print("\nDocument after clearing:")
result = r.json().get('doc', '$')
pprint(result)

As with all Redis keys, you can use the `DEL` command to delete keys entirely.

In [ ]:
# Delete the doc key entirely
response = r.delete('doc')
print(f"Key deleted: {response}")

# Try to get the document (should fail)
try:
    result = r.json().get('doc', '$')
    print(result)
except Exception as e:
    print(f"Error: {e}")